In [1]:
import json
import os
from pathlib import Path

In [2]:
indir = "../data/old_prompts"
outdir = "../data/prompts"

In [25]:
def adapt_instance_dict(instance):
    keys_to_change_mapping = {"Adversarial" : "Regular-Prompt",
                              "Pseudo-Adversarial" : "Hint-Prompt",
                              "CoT-Adversarial" : "CoT-Prompt",
                              "Adversarial-CoT" : "Regular-Prompt-CoT",
                              "Pseudo-Adversarial-CoT" : "Hint-Prompt-CoT"}
    adapted_instance = dict()
    for key,value in instance.items():
        if key in keys_to_change_mapping.keys():
            adapted_instance[keys_to_change_mapping[key]] = value
        else:
            adapted_instance[key] = value
    return adapted_instance

In [26]:
for subdir, dirs, files in os.walk(indir):
    for file in files:
        curr_outdir = subdir.replace(indir, outdir)
        path = Path(curr_outdir)
        path.mkdir(parents=True, exist_ok=True)
        with open(os.path.join(subdir, file), 'r') as f1:
            curr_data = json.loads(f1.read())
        if "zero_shot" in subdir:
            curr_updated_data = {variant:{instance_type:[adapt_instance_dict(instance) for instance in instances] for instance_type,instances in instance_types.items()} for variant,instance_types in curr_data.items()}
        else:
            curr_updated_data = {variant:{icl_examples_variant:{instance_type:[adapt_instance_dict(instance) for instance in instances] for instance_type,instances in instance_types.items()} for icl_examples_variant,instance_types in icl_examples_variants.items()} for variant,icl_examples_variants in curr_data.items()}    
        
        with open(os.path.join(curr_outdir, file), 'w') as f2:
            f2.write(json.dumps(curr_updated_data, indent=2))